# Lux AI 2021 python game engine performance comparison to original
See https://github.com/glmcdona/LuxPythonEnvGym for environment project and updates.

This is a python replica of the Lux game engine to speed up training and allow better game control for RL models. In this notebook we run a simple performance benchmark comparing the game engine performance of the two.

In [ ]:
!pip install kaggle-environments -U

In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .

## Test the original LuxAI2021 engine (TypeScript driven)

In [ ]:
import time
from kaggle_environments import make

from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    # No logic.
    
    return actions


# Play N games to measure performance
start_time = time.time()
total_steps = 0
num_episodes = 2000

for i in range(num_episodes):
    env = make("lux_ai_2021", debug=False) # Note, this line is needed here to reset the environment with a new seed. It doesn't effect performance significantly.
    steps = env.run([agent, agent])
    total_steps += len(steps)

total_time = time.time() - start_time

print("Original game engine: %.3f seconds per full game, mean of %.1f steps per game." % (total_time / num_episodes, total_steps / num_episodes))


## Test the python-engine replica game performance

In [ ]:
!pip install git+https://github.com/glmcdona/LuxPythonEnvGym.git@main

In [ ]:
import time
from luxai2021.game.game import Game
from luxai2021.game.actions import *
from luxai2021.game.constants import LuxMatchConfigs_Default

# Create a game
configs = LuxMatchConfigs_Default
game = Game(configs)

# Play N games to measure performance
start_time = time.time()
total_steps = 0
num_episodes = 2000
for i in range(num_episodes):
    gameOver = False
    while not gameOver:
        gameOver = game.run_turn_with_actions(actions=[])
    
    total_steps += game.state["turn"]
    game.reset()
total_time = time.time() - start_time

print("Python replica game engine: %.3f seconds per full game, mean of %.1f steps per game." % (total_time / num_episodes, total_steps / num_episodes))
